# Video Streams
## Computer Vision and Image Processing - Lab Session 5 - Exercises Solutions
### Prof: Luigi Di Stefano, luigi.distefano@unibo.it
### Tutor: Pierluigi Zama Ramirez, pierluigi.zama@unibo.it - Riccardo Spezialetti, riccardo.spezialetti@unibo.it

## Exercise 0: Distance Functions between images

Define three functions to calculate the $L_1$, $L_2$ and $L_{\infty}$ distances between two images.

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

def l1_distance(img1, img2):
    diff = np.abs(img1-img2)
    print(len(img1.shape))
    if img1.shape[-1] ==  3 and len(img1.shape)==3:
        diff = np.sum(diff, axis=-1)
    return diff

def l2_distance(img1, img2):
    sq_dist = (img1-img2)**2
    if img1.shape[-1] ==  3 and len(img1.shape)==3:
        sq_dist = np.sum(sq_dist,axis=-1)
    diff = np.sqrt(sq_dist)
    return diff

def max_distance(img1, img2):
    diff = np.abs(img1-img2)
    if img1.shape[-1] ==  3 and len(img1.shape)==3:
        diff = np.max(diff, axis=-1)
    return diff

In [ ]:
# Defining an utility to visualize change detection images and masks
def display_image_and_mask(img, mask):
    img_copy = img.copy()
    img_copy[np.logical_not(mask)] = np.asarray([255,255,255])
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.axis('off')
    plt.imshow(mask.astype(np.uint8)*255, cmap='gray', vmin=0, vmax=255)
    plt.subplot(1,2,2)
    plt.axis('off')
    plt.imshow(cv2.cvtColor(img_copy.astype(np.uint8), cv2.COLOR_BGR2RGB))
    plt.show()

## Exercise 1: Two Frame Difference

Calculate a Two Frame Difference over frames of a video to detect motion changes. Try different distance functions and visualize results. Test it on "ex/1.avi".

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
# Import additional library to properply play videos on jupyter notebook
from IPython.display import clear_output

# Defining a TFD based on a distance function and a threshold
def two_frame_difference(frame, previous_frame, distance=l1_distance, th=5):
    dist = distance(frame, previous_frame)
    mask = dist > th
    display_image_and_mask(frame,mask)

# Assigning to variable distance a distance function. Change this variable to test other distance functions
distance = l1_distance

# Defining a threshold
threshold = 30

# Loading video
idx = 0
cap = cv2.VideoCapture('ex/1.avi')

# Computing a TFD over frames
try:
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret and not frame is None:
            if idx == 0:
                # If first frame initializing  previous frame
                previous_frame = frame.astype(float)
            else:
                frame = frame.astype(float)
                # Calculating a TFD
                two_frame_difference(frame, previous_frame, distance, threshold)
                clear_output(wait=True)
                # Updating previous frame
                previous_frame = frame
            idx += 1
        else:
            break
    cap.release()
except KeyboardInterrupt:
    # If we press top release the video
    cap.release()
    print("Released Video Resource")

## Exercise 2: Three Frame Difference

Calculate a Three Frame Difference over frames of a video to detect motion changes. Try different distance functions and visualize results. Test it on "ex/1.avi".

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
# Import additional library to properply play videos on jupyter notebook
from IPython.display import clear_output

def two_frame_difference(frame, previous_frame, distance=l1_distance, th=5):
    dist = distance(frame, previous_frame)
    mask = dist > th
    return mask

def three_frame_difference(frame, previous_frames, distance=l1_distance, th=5):
    masks = []
    masks.append(two_frame_difference(frame, previous_frames[1], distance, th))
    if len(previous_frames) > 1:
        masks.append(two_frame_difference(previous_frames[1], previous_frames[0], distance, th))
    and_mask = np.prod(masks,axis=0)
    # mask relative to frame t-1 ! Display the and mask with previous_frame[1]
    display_image_and_mask(previous_frames[1], and_mask)

# Assigning to variable distance the function l1_distance
distance = l1_distance
threshold = 30

idx = 0
cap = cv2.VideoCapture('ex/1.avi')
previous_frames = []

try:
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret and not frame is None:
            if idx < 2 :
                # initialize previous frames properly
                previous_frames.append(frame.astype(float))
            else:
                frame = frame.astype(float)
                three_frame_difference(frame, previous_frames, distance, threshold)
                clear_output(wait=True)
                # update previous frames
                previous_frames.pop(0)
                previous_frames.append(frame)
            idx += 1
        else:
            break
    cap.release()
except KeyboardInterrupt:
    # If we press top release the video
    cap.release()
    print("Released Video Resource")

## Exercise 3: Background Subtraction

Apply a background subtraction to all frames of a video to detect motion changes. 

To initialize a background image try:
* Using only the first frame 
* Finding the mean or median over the first $n$ frames.

Test it on "ex/1.avi".

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
# Import additional library to properply play videos on jupyter notebook
from IPython.display import clear_output

# Defining a variable interpolation for mean or median functions
interpolation = np.median # or np.mean

# Loading Video
num_frame_bg = 100
bg = []
idx = 0
cap = cv2.VideoCapture('ex/1.avi')

# Initialize the background image
try:
    while(cap.isOpened() and idx < num_frame_bg):
        ret, frame = cap.read()
        if ret and not frame is None:
            frame = frame.astype(float)
            # Getting all first n images
            bg.append(frame)
            idx += 1
        else:
            break
    cap.release()
    
    # Finding the background as the mean or median of first n frames
    bg_interpolated = np.stack(bg, axis=0)
    # Median
    bg_interpolated = interpolation(bg_interpolated, axis=0)
    
    plt.title("Background")
    plt.imshow(cv2.cvtColor(bg_interpolated.astype(np.uint8), cv2.COLOR_BGR2RGB))
    plt.show()
except KeyboardInterrupt:
    # If we press top release the video
    cap.release()
    print("Released Video Resource")

In [ ]:
# Loading Video
threshold = 20
cap = cv2.VideoCapture('ex/1.avi')
distance = l1_distance
# Background subtraction
try:
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret and not frame is None:
            frame = frame.astype(float)
            mask = distance(frame, bg_interpolated) > threshold
            display_image_and_mask(frame, mask)
            clear_output(wait=True)
        else:
            break
    cap.release()
except KeyboardInterrupt:
    # If we press top release the video
    cap.release()
    print("Released Video Resource")

## Exercise 4: Sobel Filter Frame-Wise

For each frame of video "ex/1.avi", apply the Sobel kernels to calculate the smooth derivates $\frac{dI(x, y)}{dx}$, $\frac{dI(x, y)}{dy}$ along x and y respectively. Then, calculate the module of the gradient as $max(abs(\frac{dI(x, y)}{dx}), abs(\frac{dI(x, y)}{dy}))$. Visualize the results.

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
# Import additional library to properply play videos on jupyter notebook
from IPython.display import clear_output


# Defining sobel kernels
sobel_kernel_x = np.array([
            [-1,0,1],
            [-2,0,2],
            [-1,0,1]])*1/4

sobel_kernel_y = np.array([
            [-1,-2,-1],
            [0,0,0],
            [1,2,1]])*1/4

def edge_sobel(img):
    img = img.astype(float)
    # Finding dI(x, y)/dx
    dx = np.abs(cv2.filter2D(img,-1,sobel_kernel_x))
    # Finding dI(x, y)/dy
    dy = np.abs(cv2.filter2D(img,-1,sobel_kernel_y))
    # Finding gradient module pixel-wise
    sobel = np.maximum(dx,dy)
    return sobel

cap = cv2.VideoCapture('ex/1.avi')

try:
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret and not frame is None:
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            edges = edge_sobel(frame_gray)
            plt.axis('off')
            plt.imshow(edges, cmap='gray', vmin=0, vmax=255)
            plt.show()
            clear_output(wait=True)
        else:
            break
    cap.release()
except KeyboardInterrupt:
    # If we press top release the video
    cap.release()
    print("Released Video Resource")